# Circuit Telemetry Analysis & Visualization

This notebook analyzes Assetto Corsa telemetry data from any circuit and creates comprehensive track visualizations for AI training support.

### Key Features:
- **Circuit-agnostic analysis** - Works with any AC track (Monza, Silverstone, etc.)
- **Automatic track optimization** - Rotates and scales circuit for optimal display
- **Multi-variable visualization** - 6 different telemetry views in interactive plots

### Analysis Components:

1. **Data Loading & Inspection**
   - Telemetry data overview (coordinates, speed, distance)
   - Available variables summary (steering, grip, flags, etc.)

2. **Track Layout Optimization** 
   - Automatic rotation for best horizontal fit
   - Coordinate centering and scaling

3. **Interactive Visualizations** (Plotly)
   - **Speed map** - Track colored by speed zones
   - **Steering map** - Cornering analysis 
   - **Throttle/Brake map** - Acceleration/braking zones
   - **Gear map** - Transmission usage patterns
   - **RPM map** - Engine performance zones
   - **Surface Grip map** - Track condition analysis

### Variables Used:
`Speed_kmh`, `RPM`, `Throttle`, `Brake`, `Steering`, `Gear`, `SurfaceGrip`, `X`, `Y`, `Z`

### Output:
Interactive multi-subplot visualization showing complete telemetry analysis for driving pattern recognition and AI training data preparation.

---

## 1. Import Libraries and Data Loading

Import essential libraries for telemetry analysis and visualization

- **pandas/numpy**: Data manipulation and mathematical operations
- **matplotlib**: Static plotting and animation capabilities
- **plotly**: Interactive visualizations for detailed telemetry analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#### 1.1 📊 Telemetry Data Loading & Inspection
**Purpose**: Load AC telemetry data and analyze available variables for circuit analysis
- **Data overview**: Check coordinate ranges, speed limits, and total distance
- **Variable discovery**: Identify new telemetry fields (steering, grip, flags)
- **Quality assessment**: Validate data completeness for visualization pipeline

In [8]:
# Load telemetry data
df = pd.read_csv('../TELEMETRY/LAPS_OUTPUT/lap_2_telemetry.csv')

# Analyze available coordinates
print("=== MONZA ASSETTO CORSA DATA ===")
print(f"Total points: {len(df)}")
print(f"Coordinates - X: {df['X'].min():.1f} to {df['X'].max():.1f}")
print(f"Coordinates - Y: {df['Y'].min():.1f} to {df['Y'].max():.1f} (height)")  
print(f"Coordinates - Z: {df['Z'].min():.1f} to {df['Z'].max():.1f}")
print(f"Speed: {df['Speed_kmh'].min():.1f} - {df['Speed_kmh'].max():.1f} km/h")
print(f"Total distance: {df['Distance'].max():.1f} m")


df.columns

=== MONZA ASSETTO CORSA DATA ===
Total points: 814
Coordinates - X: -212.2 to 1045.4
Coordinates - Y: -11.3 to 1.6 (height)
Coordinates - Z: -1276.7 to 889.5
Speed: 77.1 - 335.0 km/h
Total distance: 5748.6 m


Index(['Distance', 'Timestamp', 'Speed_kmh', 'RPM', 'Throttle', 'Brake',
       'Steering', 'Gear', 'CompletedLaps', 'iCurrentTime_ms',
       'CurrentLapTime_str', 'iLastTime_ms', 'iBestTime_ms', 'LapNumberTotal',
       'CurrentSectorIndex', 'LastSectorTime_ms', 'IsInPit', 'IsInPitLane',
       'TyreCompound', 'X', 'Y', 'Z', 'Flag', 'SurfaceGrip'],
      dtype='object')

In [3]:
# New available variables
print(f"\n=== NEW AVAILABLE VARIABLES ===")
print(f"Steering: {df['Steering'].min():.3f} to {df['Steering'].max():.3f}")
print(f"SurfaceGrip: {df['SurfaceGrip'].unique()}")
print(f"TyreCompound: {df['TyreCompound'].unique()}")
print(f"Flag: {df['Flag'].unique()}")


=== NEW AVAILABLE VARIABLES ===
Steering: -0.879 to 0.782
SurfaceGrip: [1.]
TyreCompound: ['Soft (S)']
Flag: [0]


---

## 2. Coordinate optimization 

#### 2.1 🔧 Circuit Coordinate Transformation Functions

**Purpose**: Implement geometric utilities to optimize circuit layout visualization through coordinate transformations.

**What will be accomplished:**
- Center coordinates to eliminate positioning offsets
- Apply 2D rotations for optimal track orientation
- Calculate aspect ratios to maximize horizontal display space

**Function Details:**

**`center_coordinates(x, z)`**: Translates coordinates to origin (0,0) by subtracting mean values. Essential for subsequent rotation operations.

**`rotate_coordinates(x, z, angle_rad)`**: Applies 2D rotation matrix using standard trigonometric formulas to test different circuit orientations.

**`calculate_aspect_ratio(x, z)`**: Computes width/height ratio to identify orientations that best utilize horizontal screen space. Includes division-by-zero protection.

In [4]:
def center_coordinates(x, z):
    """
    Center coordinate arrays around their mean values.
    
    Args:
        x (np.array): X coordinate values
        z (np.array): Z coordinate values
        
    Returns:
        tuple: Centered (x, z) coordinates as numpy arrays
    """
    return x - np.mean(x), z - np.mean(z)

def rotate_coordinates(x, z, angle_rad):
    """
    Apply 2D rotation transformation to coordinate arrays.
    
    Args:
        x (np.array): X coordinate values
        z (np.array): Z coordinate values
        angle_rad (float): Rotation angle in radians
        
    Returns:
        tuple: Rotated (x_rot, z_rot) coordinates as numpy arrays
    """
    x_rot = x * np.cos(angle_rad) - z * np.sin(angle_rad)
    z_rot = x * np.sin(angle_rad) + z * np.cos(angle_rad)
    return x_rot, z_rot

def calculate_aspect_ratio(x, z):
    """
    Calculate the width-to-height aspect ratio of coordinate bounds.
    
    Args:
        x (np.array): X coordinate values
        z (np.array): Z coordinate values
        
    Returns:
        float: Aspect ratio (width/height) with small epsilon to avoid division by zero
    """
    width = np.max(x) - np.min(x)
    height = np.max(z) - np.min(z)
    return width / (height + 0.001)

#### 2.2 🔄 Circuit Layout Optimization Algorithm

**Purpose**: Automatically determine the optimal circuit orientation for horizontal display by testing multiple rotation angles.

**What will be accomplished:**
- Center the circuit coordinates around origin
- Test 18 different rotation angles (0° to 170° in 10° increments)
- Find the rotation that maximizes horizontal aspect ratio
- Return optimally oriented coordinates

**Algorithm Details:**

**Initialization**: Centers coordinates and sets tracking variables for best rotation angle and aspect ratio.

**Rotation Testing Loop**: Iterates through angles, applies rotation transformation, calculates aspect ratio, and tracks the configuration with highest width-to-height ratio.

**Output**: Returns the optimally rotated coordinates and prints the best rotation angle with its aspect ratio for verification.

In [5]:
def optimize_circuit_layout(track_x, track_z):
    """
    Find optimal circuit orientation by maximizing aspect ratio through rotation.
    
    Tests different rotation angles (0-180 degrees) to find the orientation
    that maximizes the track's width-to-height ratio for better visualization.
    
    Args:
        track_x (np.array): Raw X coordinate values from telemetry
        track_z (np.array): Raw Z coordinate values from telemetry
        
    Returns:
        tuple: Optimized (x, z) coordinates with best orientation
    """
    
    # Center coordinates
    x_centered, z_centered = center_coordinates(track_x, track_z)
    
    # Find best rotation angle
    best_ratio = 0
    best_rotation = 0
    best_coords = (x_centered, z_centered)
    
    for angle_deg in range(0, 180, 10):
        angle_rad = np.radians(angle_deg)
        x_rot, z_rot = rotate_coordinates(x_centered, z_centered, angle_rad)
        ratio = calculate_aspect_ratio(x_rot, z_rot)
        
        if ratio > best_ratio:
            best_ratio = ratio
            best_rotation = angle_deg
            best_coords = (x_rot, z_rot)
    
    print(f"Best rotation: {best_rotation}°, ratio: {best_ratio:.2f}")
    return best_coords

#### 2.3 📊 Data Preparation & Layout Optimization Execution

**Purpose**: Extract coordinate data from telemetry DataFrame, apply optimization algorithm, and prepare variables for visualization.

**What will be accomplished:**
- Extract X, Z coordinates from telemetry data (AC coordinate system: Z is horizontal)
- Apply the optimization algorithm to find best circuit orientation
- Prepare telemetry variables dictionary for subsequent plotting functions

**Implementation:**
- **Coordinate extraction**: Gets raw X,Z values; Y (height) extracted but unused for 2D maps
- **Layout optimization**: Calls `optimize_circuit_layout()` to get centered and rotated coordinates
- **Variable preparation**: Creates dictionary with key telemetry metrics for visualization pipeline

In [ ]:
def prepare_circuit_data(df):
    """
    Extract track coordinates from telemetry data and optimize layout.
    
    Processes the X and Z position data from the loaded telemetry DataFrame,
    applies coordinate optimization for better visualization, and prepares
    telemetry variables for multi-variable analysis.
    
    Args:
        df (pd.DataFrame): Telemetry data with position and vehicle metrics
        
    Returns:
        tuple: (track_x, track_z, telemetry_vars) - Optimized coordinates and variables dict
    """
    # Extract coordinates (AC coordinate system: Z is horizontal)
    coords_raw = {
        'x': df['X'].values,
        'z': df['Z'].values,  
        'y': df['Y'].values  # Height (unused for 2D maps)
    }
    
    # Optimize layout for horizontal display
    track_x, track_z = optimize_circuit_layout(coords_raw['x'], coords_raw['z'])
    
    # Prepare telemetry variables for visualization
    telemetry_vars = {
        'speeds': df['Speed_kmh'].values,
        'steering': df['Steering'].values,
        'surface_grip': df['SurfaceGrip'].values
    }
    
    return track_x, track_z, telemetry_vars

#### 2.4 🚀 Execute Circuit Data Preparation

**Purpose**: Apply the complete coordinate optimization pipeline to the loaded telemetry data.

**What will be accomplished:**
- Extract X, Z coordinates from the Monza telemetry DataFrame
- Apply automatic layout optimization to find best horizontal orientation
- Prepare telemetry variable arrays for visualization pipeline
- Store optimized coordinates and variables for subsequent plotting functions

In [7]:
# Execute data preparation
track_x, track_z, telemetry_vars = prepare_circuit_data(df)

NameError: name 'track_x' is not defined

In [ ]:
def create_track_trace(x, y, data, colorscale, name, precision=1):
    """
    Create a single track trace with telemetry data visualization.
    
    Args:
        x (np.array): X coordinates for track path
        y (np.array): Y coordinates for track path  
        data (np.array): Telemetry data values for color mapping
        colorscale (str): Plotly colorscale name for data visualization
        name (str): Display name for the trace
        precision (int): Decimal precision for hover display
        
    Returns:
        go.Scatter: Plotly scatter trace with color-coded telemetry data
    """
    return go.Scatter(
        x=x, y=y, mode='markers+lines',
        marker=dict(color=data, colorscale=colorscale, size=4),
        line=dict(width=1, color='rgba(0,0,0,0.3)'),
        name=name,
        hovertemplate=f'{name}: %{{marker.color:.{precision}f}}<br>X: %{{x:.0f}}<br>Z: %{{y:.0f}}'
    )

def create_subplot_grid():
    """
    Create empty subplot grid with titles for telemetry visualization.
    
    Returns:
        plotly.subplots: 3x2 subplot grid with predefined titles
    """
    return make_subplots(
        rows=3, cols=2,
        subplot_titles=('Speed', 'Steering', 'Throttle/Brake', 'Gear', 'RPM', 'Surface Grip'),
        specs=[[{"type": "scatter"}] * 2] * 3
    )

def setup_subplot_layout(fig):
    """
    Configure axes and layout properties for all subplots.
    
    Args:
        fig (plotly.graph_objects.Figure): Figure object with subplots to configure
    """
    fig.update_layout(title='AC Circuit - Complete Telemetry Analysis', showlegend=False, height=1000)
    
    for row in range(1, 4):
        for col in range(1, 3):
            fig.update_xaxes(title_text="X Position (m)", row=row, col=col)
            fig.update_yaxes(title_text="Z Position (m)", row=row, col=col, 
                           scaleanchor=f"x{(row-1)*2+col}", scaleratio=1)

def get_telemetry_configs(df, telemetry_vars):
    """
    Define telemetry data configurations for multi-variable visualization.
    
    Args:
        df (pd.DataFrame): Telemetry data containing vehicle metrics
        telemetry_vars (dict): Dictionary with prepared telemetry arrays
        
    Returns:
        list: List of tuples containing (data, colorscale, name, precision) for each plot
    """
    return [
        (telemetry_vars['speeds'], 'Viridis', 'Speed', 1),
        (telemetry_vars['steering'], 'RdBu', 'Steering', 3),
        (df['Throttle'] - df['Brake'], 'RdYlGn', 'T/B', 2),
        (df['Gear'], 'Plasma', 'Gear', 0),
        (df['RPM'], 'Hot', 'RPM', 0),
        (telemetry_vars['surface_grip'], 'Blues', 'Surface Grip', 2)
    ]

---

## 3. Interactive Visualization Functions

#### 3.1 📊 Plotly Track Visualization Components

**Purpose**: Implement modular functions to create interactive multi-variable telemetry visualizations using Plotly.

**What will be accomplished:**
- **Track trace creation**: Generate individual track plots with color-coded telemetry data
- **Subplot grid setup**: Create organized 3x2 layout for six different telemetry aspects  
- **Layout configuration**: Apply consistent styling, axis scaling, and hover interactions
- **Telemetry data mapping**: Define configurations for speed, steering, throttle/brake, gear, RPM, and surface grip analysis

**Function Overview:**
- `create_track_trace()`: Individual trace with color mapping and hover details
- `create_subplot_grid()`: 3x2 subplot framework with predefined titles
- `setup_subplot_layout()`: Axis scaling, labels, and visual styling
- `get_telemetry_configs()`: Data and colorscale definitions for each telemetry variable

In [ ]:
def create_circuit_telemetry_visualization(df, track_x, track_z, telemetry_vars):
    """
    Create comprehensive interactive multi-variable circuit telemetry visualization.
    
    Generates a 3x2 subplot grid showing six different telemetry aspects:
    speed zones, steering analysis, throttle/brake patterns, gear usage,
    RPM distribution, and surface grip conditions.
    
    Args:
        df (pd.DataFrame): Complete telemetry data with vehicle metrics
        track_x (np.array): Optimized X coordinates for track layout
        track_z (np.array): Optimized Z coordinates for track layout  
        telemetry_vars (dict): Dictionary containing prepared telemetry arrays
    """
    
    fig = create_subplot_grid()
    telemetry_configs = get_telemetry_configs(df, telemetry_vars)
    positions = [(1,1), (1,2), (2,1), (2,2), (3,1), (3,2)]
    
    # Add all traces
    for (data, colorscale, name, precision), (row, col) in zip(telemetry_configs, positions):
        trace = create_track_trace(track_x, track_z, data, colorscale, name, precision)
        fig.add_trace(trace, row=row, col=col)
    
    setup_subplot_layout(fig)
    fig.show()

#### 3.2 🎯 Complete Multi-Variable Visualization Pipeline

**Purpose**: Orchestrate the creation of comprehensive interactive telemetry analysis with six synchronized track views.

**What will be accomplished:**
- **Speed analysis**: Color-coded track showing velocity zones and racing lines
- **Steering analysis**: Track colored by steering input intensity for cornering evaluation  
- **Throttle/Brake patterns**: Combined visualization showing acceleration and braking zones
- **Gear usage mapping**: Track segments colored by transmission gear for performance analysis
- **RPM distribution**: Engine performance zones across the circuit layout
- **Surface grip conditions**: Track surface quality analysis for setup optimization

**Visualization Features:**
- Interactive hover details with precise telemetry values
- Synchronized axis scaling across all subplots for consistent spatial reference
- Color-coded legends optimized for each telemetry variable type
- Comprehensive title and layout styling for professional analysis presentation

In [ ]:
# Create visualization
create_circuit_telemetry_visualization(df, track_x, track_z, telemetry_vars)

#### 3.3 🏁 Execute Complete Telemetry Visualization

**Purpose**: Generate the final interactive multi-variable circuit analysis visualization.

**Expected Output:**
- **Six synchronized track views** showing different telemetry aspects
- **Interactive hover functionality** with precise data values at each track position  
- **Professional layout** suitable for racing analysis and AI training data preparation
- **Comprehensive telemetry coverage** for complete driving pattern recognition

**Analysis Applications:**
- **Racing line optimization**: Speed and steering pattern analysis
- **Vehicle setup insights**: Gear usage and RPM distribution evaluation
- **Track surface analysis**: Surface grip condition mapping for tire strategy
- **AI training preparation**: Multi-dimensional telemetry data for machine learning models